# **CS825-001, Spring/Summer 2020**

## **Assignment 1 - Question 2**

### **Assigned Date: Saturday, May 16, 2020**

### **Due Date: Tuesday, May 28, 2020**


# **Question - 1**

### Write a program that reads a grayscale image in raw format from a file; resize the image to a specified resolution using the **weighted average resampling method**; and save the new image into a new file in raw format. 

1. Input image filename: “**rose.raw**”
   *   Format: **grayscale**
   *   Original resolution: **256x256**
   *   New resolution: **500x500**


2. Input image filename: “**winter-landscape-1600x1000-grayscale.raw**”

    *  Format: **grayscale**
    *  Original resolution: **1600x1000**
    *  New resolution: **1000x625**


3. Input image filename: “**winter-landscape-1600x1000-grayscale.raw**”

    *  Format: **grayscale**
    *  Original resolution: **1600x1000**
    *  New resolution: **640x400**





---



# **Solution**

# **Input Functions**

Below are some of the functions that take the input from the user such as the resolution (rows, columns) and filename of the image.


### **Importing Libraries**

Below are some of the functions that take the input from the user such as the resolution (rows, columns) and filename of the image.


* The only reason that `numpy` library is used to convert the raw image into **array of pixels**. Mainly used to **store the values in an array**.

* The only reason that `more_itertools` is used because of its `numeric_range` function. A extension of the built-in `range()` function whose arguments can be any orderable numeric type. With only stop specified, start defaults to 0 and step defaults to 1. The output items will match the type of stop. To be more precise, this library helps the for loop to loop around decimal numbers.

In [0]:
# Necessary libraries used for this assignment
import numpy as np
import more_itertools as mit

### **Defining all the input functions**

In [0]:
# Function: Resolution
#
#    Description:  This functions takes the input from the user. 
#                  User should enter the number of rows, columns of the old image and the new image.
#
#    Parameter:    None
#
#    Result:       returns the value of the inputted rows and columns of the old and new image.
#   
#    Author:       Tanu Nanda Prabhu

def resolution():

    # Explicitly typed-casted to intergers, because the rows and columns of the images must an integer. 
    ROWS = int(input("Enter the number of rows of the old image: "))
    COLS = int(input("Enter the number of columns of the old image: "))

    ROWSNEW = int(input("Enter the number of rows of the new image: "))
    COLSNEW = int(input("Enter the number of columns of the new image: "))

    return ROWS, COLS, ROWSNEW, COLSNEW

In [0]:
# Function: FileName
#
#    Description:  This functions takes the input from the user. 
#                  User should enter the filename along with the path to read and save.
#
#    Parameter:    None
#
#    Result:       returns the name of the file to read and save.
#   
#    Author:       Tanu Nanda Prabhu

def fileName():
 
    filename_to_read = input("Enter the file name to read:  ")
    filename_to_save = input("Enter the file name to save:  ")
    return filename_to_read, filename_to_save

In [0]:
# Function: main
#
#    Description:  This is the main function of the program
#                  On executing this function, all the functions declared inside will be called. Just like in C, C++, etc.
#
#    Parameter:    None
#
#    Result:       Calls every function declared inside it.
#   
#    Author:       Tanu Nanda Prabhu

if __name__ == "__main__":

    resolution = resolution()    # Calling the resolution() and storing the result in resolution
    fileName = fileName()        # Calling the fileName() and storing the result in fileName



---



# **Reading a grayscale image in raw format from a file.**

In [30]:
# Initialising rows and column values for the image
ROWS = resolution[0]        # This are hardcoded values for the image resolution
COLS =  resolution[1]       # This particular image would contain (256*256) meaning 65,536 pixels

# Opening the input image file (RAW format)
fin = open(fileName[0])                        # Calling a filename which returns the name of 
print(fin)

<_io.TextIOWrapper name='winter-landscape-1600x1000-grayscale.raw' mode='r' encoding='UTF-8'>


Now we need to construct an 2D array from the raw data (image). A highly efficient way of reading binary data with a know data-type must be used, this can be done with the help of **numpy library**. 

In [31]:
# Loading the input image
print("... Load input image")
img = np.fromfile(fin, dtype = np.uint8, count = ROWS * COLS)
print("Old dimemsion of the array:", img.ndim)
print(img.size)

... Load input image
Old dimemsion of the array: 1
1600000


There is one way that we can convert the 1D array to 2D array such as floor dividing the total number of pixels with rows and columns of the image or columns and columns (either is fine). This can be written with the help of a formula :

`img = tp // (cols, cols)`

**tp** = total pixels; **rows** and **cols** represents rows and coloumns of the image. The reason we are using floor divison rather than division because we need to round off the values as a whole number.



In [32]:
# Conversion from 1D to 2D array
img.shape = (img.size // COLS, COLS)
print("New dimension of the array:", img.ndim)
print("----------------------------------------------------")
print(" The 2D array of the original image is: \n", img)
print("----------------------------------------------------")
print("The shape of the original image array is: ", img.shape)

New dimension of the array: 2
----------------------------------------------------
 The 2D array of the original image is: 
 [[240 240 240 ... 230 230 230]
 [240 240 239 ... 230 229 229]
 [240 239 238 ... 231 232 232]
 ...
 [ 50  50  50 ...  50  50  50]
 [ 50  50  50 ...  50  50  50]
 [ 50  50  50 ...  50  50  50]]
----------------------------------------------------
The shape of the original image array is:  (1000, 1600)


In [33]:
# This is an extra step, creating a empty 2D array, with new rows and columns (height and width)
ROWSNEW = resolution[2]
COLSNEW = resolution[3]
imgNew = np.zeros([ROWSNEW, COLSNEW])

# Converting the array into integer
imgNew = imgNew.astype(int)
print("The shape of the new Image array is: ", imgNew.shape)

The shape of the new Image array is:  (400, 640)




---



# **Resizing the image to a specified resolution using weighted average resampling method**

## **Resampling the image**

In [0]:
# Function: Resampling
#
#   Parameter: img – the original image
#              N – the number of columns
#              M – the number rows
#              x, y – the position for resampling
#
#   Result:    resampled intensity value at (x,y) position

def resampling(img, N, M, x, y):

    i = int(x)   # truncation
    j = int(y)   # truncation
    
    s = (x - i)
    t = (y - j)

    # Displaying the values for inspection
    # With the help of this we can display few points around the center of the image
    # This is mainly used for debugging purpose. (Credits to my professor for providing the code)
    if(x >= (N/2-1) and x <= (N/2+1) and y >=(M/2-1) and y <= (M/2+1)):

       print(x, y, i, j, s, t)
       print("----------------------------------------------------")
       print(img[j][i], img[j][i+1], img[j+1][i], img[j+1][i+1])
       print("----------------------------------------------------")

    
    result = img[j,i] * (1.0-s) * (1.0-t) + img[j,i+1] * s * (1.0-t) + img[j+1,i] * (1.0-s) * t + img[j+1,i+1] * s * t

    # Since we don't want the intensity values in float, we can safely explicitly type cast it to intergers as shown below
    return int(result)

## **Resizing the image**

In [0]:
# Function: Image Resizing
#
#   Parameters: img – original image
#               N, M – columns and rows of the original image
#               imgNew – resized the new image
#               N1, M1 – columns and rows of the new image


def resizeImage(img, imgNew, N, M, N1, M1):
    
    K = 2.0    # Choosing the K and D values by trial and error method.
    d = 0.9
    x = 0.0    
    y = 0.0

    Dx = float(N - 1)/(N1 - 1)    # Dx, Dy are the interval lengths for resampling
    Dy = float(M - 1)/(M1 - 1)    # Double data type are not supported in python, so using float is preferable

    for k in range(M1):
        y += Dy
        for l in range(N1):
            K2 = int(K/2)
            sum = 0.0

            for s in mit.numeric_range(-K2, K2):

                for t in mit.numeric_range(-K2, K2):
                    # To avoid array index out of bound exception, since the rows and columns of original image is 256 and 256.
                    # The resampling methods tries to access the 257 rows and 257 columns and so on...
                    # So we need to manually set the thresholds and then break the condition once if the control come here and continue looping
                    if(x+K2*d >= COLS-1 or y+K2*d >= ROWS-1 ):
                        break

                    sum = sum + resampling(img, N, M, x+K2*d,y+K2*d)

            imgNew[k][l] = sum / (K * K)   
            x = x+Dx

        # If we don't reinitialize the x value to 0.0 then at the end we will get a blank black image. So doing this is compulsory
        x = 0.0
        
    # Printing the newly image array
    print(imgNew)
    

# **Testing the program**

1. Input image filename: “rose.raw”

*   Format: grayscale
*   Original resolution: 256x256
*   New resolution: 500x500



In [4]:
# Function: main
#
#    Description:  This is the main function of the program
#                  On executing this function, all the functions declared inside will be called. Just like in C, C++, etc.
#
#    Parameter:    None
#
#    Result:       Calls every function declared inside it.
#   
#    Author:       Tanu Nanda Prabhu

if __name__ == "__main__":

    resolution = resolution()    # Calling the resolution() and storing the result in resolution
    fileName = fileName()        # Calling the fileName() and storing the result in fileName

Enter the number of rows of the old image: 256
Enter the number of columns of the old image: 256
Enter the number of rows of the new image: 500
Enter the number of columns of the new image: 500
Enter the file name to read:  rose.raw
Enter the file name to save:  rose500x500-average.raw


In [11]:
resizeImage(img, imgNew, resolution[1], resolution[0], resolution[3], resolution[2])

127.12244488978 127.12244488978 127 127 0.12244488977999879 0.12244488977999879
----------------------------------------------------
133 151 135 140
----------------------------------------------------
127.12244488978 127.12244488978 127 127 0.12244488977999879 0.12244488977999879
----------------------------------------------------
133 151 135 140
----------------------------------------------------
127.12244488978 127.12244488978 127 127 0.12244488977999879 0.12244488977999879
----------------------------------------------------
133 151 135 140
----------------------------------------------------
127.12244488978 127.12244488978 127 127 0.12244488977999879 0.12244488977999879
----------------------------------------------------
133 151 135 140
----------------------------------------------------
127.63346693386818 127.12244488978 127 127 0.6334669338681778 0.12244488977999879
----------------------------------------------------
133 151 135 140
-----------------------------------------



---



2. Input image filename: “winter-landscape-1600x1000-grayscale.raw”

*  Format: grayscale
*  Original resolution: 1600x1000
*  New resolution: 1000x625




In [16]:
# Function: main
#
#    Description:  This is the main function of the program
#                  On executing this function, all the functions declared inside will be called. Just like in C, C++, etc.
#
#    Parameter:    None
#
#    Result:       Calls every function declared inside it.
#   
#    Author:       Tanu Nanda Prabhu

if __name__ == "__main__":

    resolution = resolution()    # Calling the resolution() and storing the result in resolution
    fileName = fileName()        # Calling the fileName() and storing the result in fileName

Enter the number of rows of the old image: 1000
Enter the number of columns of the old image: 1600
Enter the number of rows of the new image: 625
Enter the number of columns of the new image: 1000
Enter the file name to read:  winter-landscape-1600x1000-grayscale.raw
Enter the file name to save:  winter-landscape-1000x625-grayscale.raw


In [23]:
resizeImage(img, imgNew, resolution[1], resolution[0], resolution[3], resolution[2])

799.5996996997055 500.40000000000185 799 500 0.5996996997055248 0.4000000000018531
----------------------------------------------------
66 67 66 66
----------------------------------------------------
799.5996996997055 500.40000000000185 799 500 0.5996996997055248 0.4000000000018531
----------------------------------------------------
66 67 66 66
----------------------------------------------------
799.5996996997055 500.40000000000185 799 500 0.5996996997055248 0.4000000000018531
----------------------------------------------------
66 67 66 66
----------------------------------------------------
799.5996996997055 500.40000000000185 799 500 0.5996996997055248 0.4000000000018531
----------------------------------------------------
66 67 66 66
----------------------------------------------------
[[239 238 238 ... 232 233   0]
 [238 237 237 ... 233 233   0]
 [236 237 238 ... 232 232   0]
 ...
 [ 50  50  50 ...  50  50   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]




---



3. Input image filename: “winter-landscape-1600x1000-grayscale.raw”

*  Format: grayscale
*  Original resolution: 1600x1000
*  New resolution: 640x400

In [29]:
# Function: main
#
#    Description:  This is the main function of the program
#                  On executing this function, all the functions declared inside will be called. Just like in C, C++, etc.
#
#    Parameter:    None
#
#    Result:       Calls every function declared inside it.
#   
#    Author:       Tanu Nanda Prabhu

if __name__ == "__main__":

    resolution = resolution()    # Calling the resolution() and storing the result in resolution
    fileName = fileName()        # Calling the fileName() and storing the result in fileName

Enter the number of rows of the old image: 1000
Enter the number of columns of the old image: 1600
Enter the number of rows of the new image: 400
Enter the number of columns of the new image: 640
Enter the file name to read:  winter-landscape-1600x1000-grayscale.raw
Enter the file name to save:  winter-landscape-640x400-grayscale.raw


In [36]:
resizeImage(img, imgNew, resolution[1], resolution[0], resolution[3], resolution[2])

799.1488262910816 499.1481203007522 799 499 0.14882629108160472 0.14812030075222538
----------------------------------------------------
67 67 66 67
----------------------------------------------------
799.1488262910816 499.1481203007522 799 499 0.14882629108160472 0.14812030075222538
----------------------------------------------------
67 67 66 67
----------------------------------------------------
799.1488262910816 499.1481203007522 799 499 0.14882629108160472 0.14812030075222538
----------------------------------------------------
67 67 66 67
----------------------------------------------------
799.1488262910816 499.1481203007522 799 499 0.14882629108160472 0.14812030075222538
----------------------------------------------------
67 67 66 67
----------------------------------------------------
[[239 238 239 ... 231 233   0]
 [236 238 239 ... 232 232   0]
 [239 237 230 ... 233 231   0]
 ...
 [ 50  50  50 ...  50  50   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]




---



## **Save the new image into a new file in raw format**

In [0]:
# Save the output image
print("... Save the output image")
imgNew.astype("int8").tofile(fileName[1])
print("... File successfully saved")
fin.close()

... Save the output image
... File successfully saved




> **Note**: While entering the input. For example if the resolution of an image is **1600x1000**, then it means the **width** is **1600** and the **height** is **1000**. Similarly in the above program the **rows** would be **1000** and the **columns** would be **1600**. 







---

